In [ ]:
from utils import *
from DataParser import DataParser
from model import MyModel
from predict import predict_cat_dog
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import shutil
import glob
import cv2
import numpy as np
import zipfile
import json

In [ ]:
#!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"abdullah0susuz","key":"414ba30eea54d34edb7294627c00488c"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats
files = glob.glob('*.zip')
for file in files:
  print(file)
  with zipfile.ZipFile(file, 'r') as zip_ref:
          zip_ref.extractall('PetImages')
!unzip /content/PetImages/train.zip
!unzip /content/PetImages/test1.zip

In [ ]:
checkpoint_path = "model.h5"
train_dir = "train"
validation = 0.3
batch_size = 16
image_size = 256
epochs = 20
rate = 1
dataParser = DataParser(train_dir, validation, batch_size, image_size,rate)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    verbose=1)

In [ ]:
my_model = MyModel((image_size,image_size,3))
model = my_model.build_model()
model.compile(loss= "binary_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

history = model.fit(generate_batches(dataParser),
          steps_per_epoch=dataParser.steps_per_epoch,
          epochs=epochs,
          validation_data = generate_batches(dataParser, train=False),
          validation_steps=dataParser.validation_steps,
          callbacks=[model_checkpoint_callback])

with open('file.json', 'w') as f:
    json.dump(history.history, f)

model.save("save-16.h")

In [ ]:
history_dict = json.load(open("file.json", 'r'))
plt.plot(history_dict['val_accuracy'], label = "val_accuracy")
plt.plot(history_dict['accuracy'], label = "accuracy")
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
predict_cat_dog(model,number=27)